In [48]:
import numpy as np
from rembg import remove 
from PIL import Image 
import cv2
from matplotlib import pyplot as plt

In [73]:
# Store path of the image in the variable input_path 
input_path =  'IMG_5657.JPEG' 
  
# Processing the image 
input = Image.open(input_path) 
  
# Removing the background from the given Image 
output = remove(input) 

alpha_matte = remove(input, alpha_matting=True, only_mask=True)

# Convert to NumPy array for processing
alpha_matte_np = np.array(alpha_matte)
  
kernelSizes = [(3, 3), (5, 5), (7, 7), (15, 15)]
# loop over the kernels sizes again
for kernelSize in kernelSizes:
	# construct a rectangular kernel form the current size, but this
	# time apply a "closing" operation
	kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
	closing = cv2.morphologyEx(alpha_matte_np, cv2.MORPH_CLOSE, kernel)

# Convert back to PIL Image
refined_mask_image = Image.fromarray(closing)

# Apply refined mask to input image
output_image = Image.composite(input, Image.new("RGBA", input.size), refined_mask_image)

# # Store path of the output image in the variable output_path 
# output_path = 'E:\C programs\ 
#                Remove BackGround\gfgoutput.png' 

# #Saving the image in the given path 
# output.save(output_path) 

In [86]:
import io
from PIL import ImageOps

# Function to remove background using rembg
def remove_background(image_path):
    with open(image_path, 'rb') as img_file:
        input_image = img_file.read()
    output_image = remove(input_image)

    # # Processing the image 
    # input_image = Image.open(input_path) 

    # alpha_matte = remove(input_image, alpha_matting=True, only_mask=True)

    # # Convert to NumPy array for processing
    # alpha_matte_np = np.array(alpha_matte)
    
    # kernel = np.ones((15, 15), np.uint8) 
      
    # # defining the closing function  
    # # over the image and structuring element 
    # closing = cv2.morphologyEx(alpha_matte_np, cv2.MORPH_CLOSE, kernel)

    # # Convert back to PIL Image
    # refined_mask_image = Image.fromarray(closing)

    # # Apply refined mask to input image
    # output_image = Image.composite(input_image, Image.new("RGBA", input_image.size), refined_mask_image)

    img = Image.open(io.BytesIO(output_image)).convert("RGBA")
    return img

# Function to add a white border
def add_white_border(image, border_size):
    # Create a new image with a white background
    new_image = Image.new("RGBA", 
                          (image.width + 2 * border_size, image.height + 2 * border_size), 
                          "WHITE")
    
    # Paste the original image in the center of the new image
    new_image.paste(image, (border_size, border_size), image)

    # output_image_border = ImageOps.expand(output_image,border=300,fill='white')
    
    return new_image

# Load the image
image_path = 'IMG_5657.JPEG'
image_without_bg = remove_background(image_path)

# Add white border
border_size = 10
final_image = add_white_border(image_without_bg, border_size)

# Save the result
final_image.save('final_image.png')


In [102]:
# Load the image using OpenCV
input_path = 'IMG_5657-Photoroom.png'
image = cv2.imread(input_path)

# Convert the image from BGR to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Remove the background using rembg
output_image = remove(image_rgb)

# Convert the result back to BGR format
output_image_bgr = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)

In [107]:
from backgroundremover.bg import remove
def remove_bg(src_img_path, out_img_path):
    model_choices = ["u2net", "u2net_human_seg", "u2netp"]
    f = open(src_img_path, "rb")
    data = f.read()
    img = remove(data, model_name=model_choices[0],
                 alpha_matting=True,
                 alpha_matting_foreground_threshold=240,
                 alpha_matting_background_threshold=10,
                 alpha_matting_erode_structure_size=10,
                 alpha_matting_base_size=1000)
    f.close()
    f = open(out_img_path, "wb")
    f.write(img)
    f.close()

In [131]:
from backgroundremover.bg import remove
def remove_bg(src_img_path, out_img_path):
    model_choices = ["u2net", "u2net_human_seg", "u2netp"]
    f = open(src_img_path, "rb")
    data = f.read()
    img = remove(data, model_name=model_choices[0],
                 alpha_matting=False)
    f.close()
    f = open(out_img_path, "wb")
    f.write(img)
    f.close()
remove_bg('IMG_5657.JPEG', 'IMG_5657_3.png')

DEBUG: path to be checked: C:\Users\hanh\.u2net\u2net.pth


In [115]:
input_path = 'IMG_5657.JPEG'

model_choices = ["u2net", "u2net_human_seg", "u2netp"]

f = open(input_path, "rb")
data = f.read()

# image = cv2.imread(input_path)

# # Convert the image from BGR to RGB
# image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# # Remove the background using rembg
# output_image = remove(image_rgb)

# Convert the result back to BGR format
# output_image_bgr = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)

output_image = remove(data, model_name=model_choices[0],
                alpha_matting=True,
                alpha_matting_foreground_threshold=240,
                alpha_matting_background_threshold=10,
                alpha_matting_erode_structure_size=10,
                alpha_matting_base_size=1000)
f.close()

DEBUG: path to be checked: C:\Users\hanh\.u2net\u2net.pth


In [126]:
img_raw = cv2.imread(input_path)

In [127]:
img_raw.shape

(2048, 1536, 3)

In [129]:
2048*1536

3145728

In [121]:
img = np.asarray(output_image)

In [123]:
img.shape

(991138,)